In [9]:
from bs4 import BeautifulSoup
from lxml import etree
import requests
import re
import pandas as pd
import csv

In [5]:
import requests
from bs4 import BeautifulSoup
from concurrent.futures import ThreadPoolExecutor

# Define uma função para obter os links de uma página
def get_links(url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'
    }

    response = requests.get(url, headers=headers)
    html_content = response.text
    soup = BeautifulSoup(html_content, 'html.parser')

    matching_elements = soup.find_all('a', class_='box-texto-relacionado')
    return [element.attrs['href'] for element in matching_elements]

# Define a lista de URLs a serem requisitadas

base_url = 'https://www.cepea.esalq.usp.br/br/diarias-de-mercado/arroz-cepea-demanda-aquecida-mantem-indicador-firme.aspx?pagina='
urls = []
for i in range(815):
    url = base_url + str(i)
    urls.append(url)

# Executa as requisições em paralelo e armazena os links em um único vetor
links = []
with ThreadPoolExecutor() as executor:
    for result in executor.map(get_links, urls):
        links += result

print(len(links))

12225


In [11]:
def obter_noticias(url: str):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'
    }

    response = requests.get(url, headers=headers)
    html_content = response.text
    soup = BeautifulSoup(html_content, 'html.parser')

    dom = etree.HTML(str(soup))
    noticia = dom.xpath('//*[@id="imagenet-content"]/div[2]/div[1]/div/div/div[1]/div[1]/p')[0].text
    titulo = dom.xpath('//*[@id="imagenet-content"]/div[2]/div[1]/div/h2')[0].text
    return noticia, url, titulo

with ThreadPoolExecutor(max_workers=8) as executor:
    noticias = []
    for result, url, titulo in executor.map(obter_noticias, links):
        padrao_data = r'\b\d{1,2}/\d{2}/\d{4}\b'
        try:
            data_noticia = re.findall(padrao_data, result)[0]
        except:
            data_noticia = None
        noticias.append({'data': data_noticia, 'titulo': titulo, 'url': url, 'noticia': result})
        df = pd.DataFrame(noticias)

        df.to_csv('../../../../work/data/ingestion/RAW/noticias_cepea.csv', index=False, mode='a', header=True,  quoting=csv.QUOTE_NONNUMERIC, escapechar="\\", doublequote=False)

IndexError: list index out of range

In [22]:
df = pd.read_csv('../../../../work/data/ingestion/RAW/noticias_agricolas.csv', on_bad_lines='skip')

In [23]:
df

,data,titulo,noticia
0,13/04/2023 18:26,BC anuncia rolagem de US$15 bi em swaps que ve...,SÃO PAULO (Reuters) - O Banco Central informou...
1,13/04/2023 17:51,Petróleo cai 1% após atingir máximas de vários...,Por Shariq Khan BENGALURU (Reuters) - Os preço...
2,13/04/2023 17:18,Wall St sobe após dados de inflação alimentare...,Por Stephen Culp NOVA YORK (Reuters) - As açõe...
3,13/04/2023 17:15,Colômbia também divulga dados de produção e ex...,NaN
4,13/04/2023 17:12,Dólar recua ante real pelo 3º dia seguido sob ...,SÃO PAULO (Reuters) -O dólar à vista emplacou ...
...,...,...,...
3915,14/03/2023 14:59,Com plantio de milho mais lento dos últimos 7 ...,O Departamento de Economia Rural (Deral) da Se...
3916,14/03/2023 14:49,Guilherme Sanches Ribeiro deixa o cargo de dir...,"Foi decidido na noite de segunda-feira (13), a..."
3917,14/03/2023 14:48,Ações europeias registram maior alta diária de...,Por Ankika Biswas e Sruthi Shankar (Reuters) -...
3918,14/03/2023 14:30,Artigo/Cepea: Com forte dependência do mercado...,"* Por Mauro Osaki, Pesquisador da área de Cust..."
